# Lab-2: Deep Learning Experiments

## Tasks:
- **Task 1:** Compare activation functions (Sigmoid, Tanh, ReLU)
- **Task 2:** Compare optimizers (SGD, SGD with Momentum, Adam)
- **Task 3:** Test Batch Normalization and Dropout scenarios


### Setup & Dataset (MNIST)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

### Model Definition


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, activation='relu', use_bn=False, dropout_rate=0.0):
        super(NeuralNetwork, self).__init__()
        
        # Select activation function
        if activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'relu':
            self.activation = nn.ReLU()
        else:
            raise ValueError(f"Unknown activation: '{activation}'. Valid options are: 'sigmoid', 'tanh', 'relu'")
        
        self.use_bn = use_bn
        
        # Define layers
        self.fc1 = nn.Linear(784, 128)
        self.bn1 = nn.BatchNorm1d(128) if use_bn else None
        self.dropout1 = nn.Dropout(dropout_rate) if dropout_rate > 0 else None
        
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64) if use_bn else None
        self.dropout2 = nn.Dropout(dropout_rate) if dropout_rate > 0 else None
        
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        
        x = self.fc1(x)
        if self.use_bn:
            x = self.bn1(x)
        x = self.activation(x)
        if self.dropout1 is not None:
            x = self.dropout1(x)
        
        x = self.fc2(x)
        if self.use_bn:
            x = self.bn2(x)
        x = self.activation(x)
        if self.dropout2 is not None:
            x = self.dropout2(x)
        
        x = self.fc3(x)
        return x

### Training & Testing Functions


In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
    
    avg_loss = total_loss / len(loader)
    acc = 100. * correct / len(loader.dataset)
    return avg_loss, acc


def test(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    
    avg_loss = total_loss / len(loader)
    acc = 100. * correct / len(loader.dataset)
    return avg_loss, acc


def run_experiment(model, optimizer, epochs=10, exp_name="Experiment"):
    """Run training and testing for specified epochs"""
    criterion = nn.CrossEntropyLoss()
    
    train_losses = []
    test_losses = []
    train_accs = []
    test_accs = []
    
    print(f"\n{exp_name}")
    print("=" * 60)
    
    try:
        for epoch in range(epochs):
            train_loss, train_acc = train(model, train_loader, optimizer, criterion)
            test_loss, test_acc = test(model, test_loader, criterion)
            
            # Check for NaN values
            if np.isnan(train_loss) or np.isnan(test_loss):
                print(f"\nWarning: NaN detected at epoch {epoch+1}. Stopping training.")
                break
            
            train_losses.append(train_loss)
            test_losses.append(test_loss)
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            
            print(f"Epoch {epoch+1:2d}/{epochs}: "
                  f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
                  f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    
    except RuntimeError as e:
        print(f"\nError during training: {e}")
        print("Training stopped early.")
    
    return {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_accs': train_accs,
        'test_accs': test_accs,
        'final_test_acc': test_accs[-1] if test_accs else 0.0
    }

## Task 1: The Activation Function Challenge

Compare the training loss and accuracy curves when using:
- **Sigmoid**: Observe if the model suffers from "vanishing gradients" or slow start
- **Tanh**: Compare its speed to Sigmoid
- **ReLU**: Document why this usually leads to faster convergence


In [ ]:
# Experiment 1: Sigmoid Activation
model_sigmoid = NeuralNetwork(activation='sigmoid').to(device)
optimizer_sigmoid = optim.SGD(model_sigmoid.parameters(), lr=0.01)
results_sigmoid = run_experiment(model_sigmoid, optimizer_sigmoid, epochs=10, 
                                  exp_name="Task 1.1: Sigmoid + SGD")

In [ ]:
# Experiment 2: Tanh Activation
model_tanh = NeuralNetwork(activation='tanh').to(device)
optimizer_tanh = optim.SGD(model_tanh.parameters(), lr=0.01)
results_tanh = run_experiment(model_tanh, optimizer_tanh, epochs=10, 
                               exp_name="Task 1.2: Tanh + SGD")

In [ ]:
# Experiment 3: ReLU Activation
model_relu = NeuralNetwork(activation='relu').to(device)
optimizer_relu = optim.SGD(model_relu.parameters(), lr=0.01)
results_relu = run_experiment(model_relu, optimizer_relu, epochs=10, 
                               exp_name="Task 1.3: ReLU + SGD")

### Task 1: Analysis of Activation Functions

**Observations:**

1. **Sigmoid:**
   - Often suffers from vanishing gradient problem
   - Outputs are between 0 and 1, which can slow down learning
   - May show slower convergence especially in deeper networks

2. **Tanh:**
   - Outputs are between -1 and 1 (zero-centered)
   - Generally faster than Sigmoid due to zero-centered outputs
   - Still can suffer from vanishing gradients

3. **ReLU:**
   - No vanishing gradient problem for positive values
   - Faster convergence due to simpler gradient computation
   - Helps with sparse activation (some neurons output 0)
   - Most commonly used in modern deep learning


In [ ]:
# Plot Task 1 Results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Comparison
axes[0].plot(results_sigmoid['train_losses'], label='Sigmoid', marker='o')
axes[0].plot(results_tanh['train_losses'], label='Tanh', marker='s')
axes[0].plot(results_relu['train_losses'], label='ReLU', marker='^')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Task 1: Training Loss Comparison')
axes[0].legend()
axes[0].grid(True)

# Test Accuracy Comparison
axes[1].plot(results_sigmoid['test_accs'], label='Sigmoid', marker='o')
axes[1].plot(results_tanh['test_accs'], label='Tanh', marker='s')
axes[1].plot(results_relu['test_accs'], label='ReLU', marker='^')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Accuracy (%)')
axes[1].set_title('Task 1: Test Accuracy Comparison')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\nFinal Test Accuracies:")
print(f"Sigmoid: {results_sigmoid['final_test_acc']:.2f}%")
print(f"Tanh: {results_tanh['final_test_acc']:.2f}%")
print(f"ReLU: {results_relu['final_test_acc']:.2f}%")

## Task 2: The Optimizer Showdown

Using the best activation function (ReLU), compare different optimizers:
- **SGD**: Observe the stability of the loss
- **SGD with Momentum**: Note how it handles "bumps" in the loss landscape
- **Adam**: Observe how quickly it reaches high accuracy compared to basic SGD


In [ ]:
# Experiment 4: ReLU + SGD (without momentum)
model_sgd = NeuralNetwork(activation='relu').to(device)
optimizer_sgd = optim.SGD(model_sgd.parameters(), lr=0.01)
results_sgd = run_experiment(model_sgd, optimizer_sgd, epochs=10, 
                              exp_name="Task 2.1: ReLU + SGD")

In [ ]:
# Experiment 5: ReLU + SGD with Momentum
model_momentum = NeuralNetwork(activation='relu').to(device)
optimizer_momentum = optim.SGD(model_momentum.parameters(), lr=0.01, momentum=0.9)
results_momentum = run_experiment(model_momentum, optimizer_momentum, epochs=10, 
                                   exp_name="Task 2.2: ReLU + SGD with Momentum")

In [ ]:
# Experiment 6: ReLU + Adam
model_adam = NeuralNetwork(activation='relu').to(device)
optimizer_adam = optim.Adam(model_adam.parameters(), lr=0.001)
results_adam = run_experiment(model_adam, optimizer_adam, epochs=10, 
                               exp_name="Task 2.3: ReLU + Adam")

### Task 2: Analysis of Optimizers

**Observations:**

1. **SGD (Stochastic Gradient Descent):**
   - Simple and straightforward optimization
   - May show more oscillations in loss
   - Slower convergence compared to adaptive methods

2. **SGD with Momentum:**
   - Accelerates convergence by accumulating gradients
   - Smooths out oscillations in the loss landscape
   - Better at escaping local minima

3. **Adam (Adaptive Moment Estimation):**
   - Combines advantages of momentum and adaptive learning rates
   - Usually fastest convergence
   - Works well with default hyperparameters


In [ ]:
# Plot Task 2 Results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training Loss Comparison
axes[0].plot(results_sgd['train_losses'], label='SGD', marker='o')
axes[0].plot(results_momentum['train_losses'], label='SGD + Momentum', marker='s')
axes[0].plot(results_adam['train_losses'], label='Adam', marker='^')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Task 2: Training Loss Comparison (ReLU)')
axes[0].legend()
axes[0].grid(True)

# Test Accuracy Comparison
axes[1].plot(results_sgd['test_accs'], label='SGD', marker='o')
axes[1].plot(results_momentum['test_accs'], label='SGD + Momentum', marker='s')
axes[1].plot(results_adam['test_accs'], label='Adam', marker='^')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Accuracy (%)')
axes[1].set_title('Task 2: Test Accuracy Comparison (ReLU)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\nFinal Test Accuracies:")
print(f"SGD: {results_sgd['final_test_acc']:.2f}%")
print(f"SGD + Momentum: {results_momentum['final_test_acc']:.2f}%")
print(f"Adam: {results_adam['final_test_acc']:.2f}%")

## Task 3: Batch Normalization and Dropout

Run specific scenarios to observe the contrast:
- **WITHOUT Batch Normalization and Dropout**
- **Without BN, Dropout=0.1**
- **With BN, Dropout=0.25**


In [ ]:
# Experiment 7: Without BN, Without Dropout
model_no_bn_no_drop = NeuralNetwork(activation='relu', use_bn=False, dropout_rate=0.0).to(device)
optimizer_7 = optim.Adam(model_no_bn_no_drop.parameters(), lr=0.001)
results_no_bn_no_drop = run_experiment(model_no_bn_no_drop, optimizer_7, epochs=10, 
                                        exp_name="Task 3.1: No BN, No Dropout")

In [ ]:
# Experiment 8: Without BN, Dropout=0.1
model_no_bn_drop01 = NeuralNetwork(activation='relu', use_bn=False, dropout_rate=0.1).to(device)
optimizer_8 = optim.Adam(model_no_bn_drop01.parameters(), lr=0.001)
results_no_bn_drop01 = run_experiment(model_no_bn_drop01, optimizer_8, epochs=10, 
                                       exp_name="Task 3.2: No BN, Dropout=0.1")

In [ ]:
# Experiment 9: With BN, Dropout=0.25
model_bn_drop025 = NeuralNetwork(activation='relu', use_bn=True, dropout_rate=0.25).to(device)
optimizer_9 = optim.Adam(model_bn_drop025.parameters(), lr=0.001)
results_bn_drop025 = run_experiment(model_bn_drop025, optimizer_9, epochs=10, 
                                     exp_name="Task 3.3: With BN, Dropout=0.25")

### Task 3: Analysis of BN and Dropout

**Observations:**

1. **Without BN and Dropout:**
   - Baseline performance
   - May overfit on training data
   - No regularization

2. **Without BN, Dropout=0.1:**
   - Light regularization
   - Helps prevent overfitting
   - May slightly reduce training speed

3. **With BN, Dropout=0.25:**
   - Batch normalization helps with training stability
   - Higher dropout provides stronger regularization
   - Usually better generalization to test data


In [ ]:
# Plot Task 3 Results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training and Test Accuracy Comparison
epochs_range = range(1, 11)

axes[0].plot(epochs_range, results_no_bn_no_drop['train_accs'], label='Train - No BN, No Dropout', linestyle='-', marker='o')
axes[0].plot(epochs_range, results_no_bn_no_drop['test_accs'], label='Test - No BN, No Dropout', linestyle='--', marker='o')
axes[0].plot(epochs_range, results_no_bn_drop01['train_accs'], label='Train - No BN, Dropout=0.1', linestyle='-', marker='s')
axes[0].plot(epochs_range, results_no_bn_drop01['test_accs'], label='Test - No BN, Dropout=0.1', linestyle='--', marker='s')
axes[0].plot(epochs_range, results_bn_drop025['train_accs'], label='Train - BN, Dropout=0.25', linestyle='-', marker='^')
axes[0].plot(epochs_range, results_bn_drop025['test_accs'], label='Test - BN, Dropout=0.25', linestyle='--', marker='^')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Task 3: Training vs Test Accuracy')
axes[0].legend(fontsize=8)
axes[0].grid(True)

# Training Loss Comparison
axes[1].plot(results_no_bn_no_drop['train_losses'], label='No BN, No Dropout', marker='o')
axes[1].plot(results_no_bn_drop01['train_losses'], label='No BN, Dropout=0.1', marker='s')
axes[1].plot(results_bn_drop025['train_losses'], label='BN, Dropout=0.25', marker='^')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Training Loss')
axes[1].set_title('Task 3: Training Loss Comparison')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\nFinal Test Accuracies:")
print(f"No BN, No Dropout: {results_no_bn_no_drop['final_test_acc']:.2f}%")
print(f"No BN, Dropout=0.1: {results_no_bn_drop01['final_test_acc']:.2f}%")
print(f"With BN, Dropout=0.25: {results_bn_drop025['final_test_acc']:.2f}%")

## Comprehensive Comparison Table

Summary of all experiments conducted in this lab.


In [ ]:
# Create comparison table
comparison_data = [
    {
        'Experiment': 1,
        'Activation': 'Sigmoid',
        'Optimizer': 'SGD',
        'BN': 'No',
        'Dropout': 0.0,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_sigmoid['final_test_acc']:.2f}"
    },
    {
        'Experiment': 2,
        'Activation': 'Tanh',
        'Optimizer': 'SGD',
        'BN': 'No',
        'Dropout': 0.0,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_tanh['final_test_acc']:.2f}"
    },
    {
        'Experiment': 3,
        'Activation': 'ReLU',
        'Optimizer': 'SGD',
        'BN': 'No',
        'Dropout': 0.0,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_relu['final_test_acc']:.2f}"
    },
    {
        'Experiment': 4,
        'Activation': 'ReLU',
        'Optimizer': 'SGD + Momentum',
        'BN': 'No',
        'Dropout': 0.0,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_momentum['final_test_acc']:.2f}"
    },
    {
        'Experiment': 5,
        'Activation': 'ReLU',
        'Optimizer': 'Adam',
        'BN': 'No',
        'Dropout': 0.0,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_adam['final_test_acc']:.2f}"
    },
    {
        'Experiment': 6,
        'Activation': 'ReLU',
        'Optimizer': 'Adam',
        'BN': 'No',
        'Dropout': 0.1,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_no_bn_drop01['final_test_acc']:.2f}"
    },
    {
        'Experiment': 7,
        'Activation': 'ReLU',
        'Optimizer': 'Adam',
        'BN': 'Yes',
        'Dropout': 0.25,
        'Epochs': 10,
        'Final Test Accuracy (%)': f"{results_bn_drop025['final_test_acc']:.2f}"
    }
]

df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("COMPREHENSIVE COMPARISON TABLE")
print("="*80)
print(df.to_string(index=False))
print("="*80)
print("\nNote: Run all cells above in sequence to generate experiment results.")

## Final Visualizations

Comprehensive plots showing loss curves and accuracy for different experiments.


In [ ]:
# Comprehensive visualization of all experiments
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Task 1 - Activation Functions (Loss)
axes[0, 0].plot(results_sigmoid['train_losses'], label='Sigmoid - Train', linestyle='-', marker='o')
axes[0, 0].plot(results_sigmoid['test_losses'], label='Sigmoid - Test', linestyle='--', marker='o')
axes[0, 0].plot(results_tanh['train_losses'], label='Tanh - Train', linestyle='-', marker='s')
axes[0, 0].plot(results_tanh['test_losses'], label='Tanh - Test', linestyle='--', marker='s')
axes[0, 0].plot(results_relu['train_losses'], label='ReLU - Train', linestyle='-', marker='^')
axes[0, 0].plot(results_relu['test_losses'], label='ReLU - Test', linestyle='--', marker='^')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Task 1: Loss Curves - Activation Functions')
axes[0, 0].legend(fontsize=8)
axes[0, 0].grid(True)

# Plot 2: Task 1 - Activation Functions (Accuracy)
axes[0, 1].plot(results_sigmoid['train_accs'], label='Sigmoid - Train', linestyle='-', marker='o')
axes[0, 1].plot(results_sigmoid['test_accs'], label='Sigmoid - Test', linestyle='--', marker='o')
axes[0, 1].plot(results_tanh['train_accs'], label='Tanh - Train', linestyle='-', marker='s')
axes[0, 1].plot(results_tanh['test_accs'], label='Tanh - Test', linestyle='--', marker='s')
axes[0, 1].plot(results_relu['train_accs'], label='ReLU - Train', linestyle='-', marker='^')
axes[0, 1].plot(results_relu['test_accs'], label='ReLU - Test', linestyle='--', marker='^')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].set_title('Task 1: Accuracy - Activation Functions')
axes[0, 1].legend(fontsize=8)
axes[0, 1].grid(True)

# Plot 3: Task 2 - Optimizers (Loss)
axes[1, 0].plot(results_sgd['train_losses'], label='SGD - Train', linestyle='-', marker='o')
axes[1, 0].plot(results_sgd['test_losses'], label='SGD - Test', linestyle='--', marker='o')
axes[1, 0].plot(results_momentum['train_losses'], label='SGD+Momentum - Train', linestyle='-', marker='s')
axes[1, 0].plot(results_momentum['test_losses'], label='SGD+Momentum - Test', linestyle='--', marker='s')
axes[1, 0].plot(results_adam['train_losses'], label='Adam - Train', linestyle='-', marker='^')
axes[1, 0].plot(results_adam['test_losses'], label='Adam - Test', linestyle='--', marker='^')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].set_title('Task 2: Loss Curves - Optimizers (ReLU)')
axes[1, 0].legend(fontsize=8)
axes[1, 0].grid(True)

# Plot 4: Task 2 - Optimizers (Accuracy)
axes[1, 1].plot(results_sgd['train_accs'], label='SGD - Train', linestyle='-', marker='o')
axes[1, 1].plot(results_sgd['test_accs'], label='SGD - Test', linestyle='--', marker='o')
axes[1, 1].plot(results_momentum['train_accs'], label='SGD+Momentum - Train', linestyle='-', marker='s')
axes[1, 1].plot(results_momentum['test_accs'], label='SGD+Momentum - Test', linestyle='--', marker='s')
axes[1, 1].plot(results_adam['train_accs'], label='Adam - Train', linestyle='-', marker='^')
axes[1, 1].plot(results_adam['test_accs'], label='Adam - Test', linestyle='--', marker='^')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].set_title('Task 2: Accuracy - Optimizers (ReLU)')
axes[1, 1].legend(fontsize=8)
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Task 3 Comprehensive Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot 1: Loss curves
axes[0].plot(results_no_bn_no_drop['train_losses'], label='No BN, No Dropout - Train', linestyle='-', marker='o')
axes[0].plot(results_no_bn_no_drop['test_losses'], label='No BN, No Dropout - Test', linestyle='--', marker='o')
axes[0].plot(results_no_bn_drop01['train_losses'], label='No BN, Dropout=0.1 - Train', linestyle='-', marker='s')
axes[0].plot(results_no_bn_drop01['test_losses'], label='No BN, Dropout=0.1 - Test', linestyle='--', marker='s')
axes[0].plot(results_bn_drop025['train_losses'], label='BN, Dropout=0.25 - Train', linestyle='-', marker='^')
axes[0].plot(results_bn_drop025['test_losses'], label='BN, Dropout=0.25 - Test', linestyle='--', marker='^')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Task 3: Loss Curves - BN and Dropout Effects')
axes[0].legend(fontsize=8)
axes[0].grid(True)

# Plot 2: Accuracy curves
axes[1].plot(results_no_bn_no_drop['train_accs'], label='No BN, No Dropout - Train', linestyle='-', marker='o')
axes[1].plot(results_no_bn_no_drop['test_accs'], label='No BN, No Dropout - Test', linestyle='--', marker='o')
axes[1].plot(results_no_bn_drop01['train_accs'], label='No BN, Dropout=0.1 - Train', linestyle='-', marker='s')
axes[1].plot(results_no_bn_drop01['test_accs'], label='No BN, Dropout=0.1 - Test', linestyle='--', marker='s')
axes[1].plot(results_bn_drop025['train_accs'], label='BN, Dropout=0.25 - Train', linestyle='-', marker='^')
axes[1].plot(results_bn_drop025['test_accs'], label='BN, Dropout=0.25 - Test', linestyle='--', marker='^')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Task 3: Accuracy - BN and Dropout Effects')
axes[1].legend(fontsize=8)
axes[1].grid(True)

plt.tight_layout()
plt.show()

## Conclusion

### Summary of Findings:

**Task 1 - Activation Functions:**
- ReLU generally performs best due to avoiding vanishing gradients
- Sigmoid and Tanh can suffer from slower convergence
- ReLU is the most commonly used activation in modern deep learning

**Task 2 - Optimizers:**
- Adam typically converges fastest
- SGD with Momentum provides better stability than basic SGD
- The choice of optimizer significantly impacts training speed

**Task 3 - Regularization:**
- Batch Normalization helps with training stability
- Dropout prevents overfitting
- Combining BN and Dropout usually gives best generalization

**Best Configuration:**
- Activation: ReLU
- Optimizer: Adam
- Regularization: Batch Normalization with moderate Dropout
